In [1]:
!pip install --upgrade wandb

^C


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import wandb
import os
from typing import Any, Dict, List
import copy
import random
import wandb

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_API_KEY"] = "183c1a6a36cbdf0405f5baacb72690845ecc8573"

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

'''mnist = torchvision.datasets.MNIST(root='mnist_data', train=True,download=True, transform=transform)

trainset = torch.utils.data.DataLoader(dataset=mnist,
                                               batch_size=100,
                                               shuffle=True)
testset = torch.utils.data.DataLoader(dataset=mnist,
                                               batch_size=100,
                                               shuffle=True)'''


trainset = torchvision.datasets.CIFAR10(root='cifar_data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='cifar_data', train=False, download=True,transform=transform)

# # This is for iid data
# trainloader = torch.utils.data.DataLoader(trainset, 
#                                           batch_size=500,
#                                           shuffle=True)
testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=100,
                                        shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
n_client = 10
n_leader = n_client * 0.1
chosen_prob = 0.8
local_batch_size = 32
local_epochs = 10

In [6]:
epochs = 10
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD
optimizer_conf = dict(
    lr=0.001,
    momentum=0.9
)

In [7]:
import custom_model
import server
model = custom_model.Net().to(device)

server = server.Server(
    model=model,
    loss=criteria,
    optimizer=optimizer,
    n_client=n_client,
    #n_leader=n_leader,
    chosen_prob=chosen_prob,
    optimizer_conf=optimizer_conf,
    local_batch_size=local_batch_size,
    local_epochs=local_epochs
)

print(server.client_pool)

Requirement already up-to-date: wandb in c:\users\lloflo\anaconda3\lib\site-packages (0.12.15)


Requirement already up-to-date: wandb in c:\users\lloflo\anaconda3\lib\site-packages (0.12.15)


Requirement already up-to-date: wandb in c:\users\lloflo\anaconda3\lib\site-packages (0.12.15)
[<clients.Client object at 0x000001FB0DF39188>, <clients.Client object at 0x000001FB13F62648>, <clients.Client object at 0x000001FB0DF25D08>, <clients.Client object at 0x000001FB13F787C8>, <clients.Client object at 0x000001FB13F78F88>, <clients.Client object at 0x000001FB13F9F7C8>, <clients.Client object at 0x000001FB13F9FF88>, <clients.Client object at 0x000001FB15D7C788>, <clients.Client object at 0x000001FB15D7CF48>, <clients.Client object at 0x000001FB15D89708>]


In [8]:
from collections import Counter
import random
import numpy as np
from tqdm import tqdm

classes_pair = [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]

data_label = np.array(trainset.targets)

chosen_counter = Counter()
for _, client in tqdm(enumerate(server.client_pool)):
    # sample until we have a pair of class with insufficient client owning
    class_pair = random.choice(classes_pair)
    while chosen_counter[class_pair] == 100:
        class_pair = random.choice(classes_pair)
        
    chosen_counter[class_pair] += 1
        
    first_class, second_class = class_pair
    first_class_sample_idx = list(np.where(data_label == first_class)[0])
    second_class_sample_idx = list(np.where(data_label == second_class)[0])
    
    client_first_class_sample_idx = random.sample(first_class_sample_idx, k=500)
    client_second_class_sample_idx = random.sample(second_class_sample_idx, k=500)
    
    client_data = []
    
    for i in range(500):
        client_data.append(trainset[client_first_class_sample_idx[i]])
        client_data.append(trainset[client_second_class_sample_idx[i]])
    
    client.setData(client_data)

10it [00:01,  5.85it/s]


In [11]:
wandb.init(project="fl", name="CNN_CIFAR_10_noniid")
for i in range(epochs):
    server.aggregate()
    server.broadcast()
    total_correct = 0
    with torch.no_grad():
        for _, (test_feature, test_label) in enumerate(testloader):
            test_feature = test_feature.to(device)
            test_label = test_label.to(device)
            y_pred = server.model(test_feature)
            y_pred_decode = torch.argmax(y_pred, dim=1)
            
            total_correct += y_pred_decode.eq(test_label).sum().item()
    
    test_acc = total_correct / 10000
        
    print("Overall acc: {}, overall_loss: {}, test_acc: {}".format(server.avg_acc, server.avg_loss, test_acc))
    wandb.log({"acc": server.avg_acc, "loss": server.avg_loss, "test_acc": test_acc})

wandb: Currently logged in as: minhrongcon2000 (use `wandb login --relogin` to force relogin)


Client 2: Acc 0.529296875, Loss: 0.6831940189003944
Client 5: Acc 0.537109375, Loss: 0.6823066230863333
Client 7: Acc 0.6826171875, Loss: 0.5974577460438013
Client 9: Acc 0.654296875, Loss: 0.6089609898626804
Client 1: Acc 0.71484375, Loss: 0.5653412090614438
Client 6: Acc 0.544921875, Loss: 0.686870327219367
Client 8: Acc 0.6494140625, Loss: 0.6110292114317417
Client 3: Acc 0.6044921875, Loss: 0.6645826082676649
Overall acc: 0.6146240234375, overall_loss: 0.6374678417341784, test_acc: 0.1361
Client 1: Acc 0.7509765625, Loss: 0.5145068382844329
Client 2: Acc 0.6259765625, Loss: 0.6444483250379562
Client 0: Acc 0.7138671875, Loss: 0.5518572321161628
Client 8: Acc 0.66796875, Loss: 0.5965033983811736
Client 6: Acc 0.6103515625, Loss: 0.6632504556328058
Client 9: Acc 0.6806640625, Loss: 0.5804040273651481
Client 4: Acc 0.61328125, Loss: 0.6361487843096256
Client 7: Acc 0.748046875, Loss: 0.5024575246497989
Overall acc: 0.6763916015625, overall_loss: 0.586197073222138, test_acc: 0.1649
Cli